# Juego del 8-puzzl - A*

### Estudiante: Rebeca Justiniano Saravia

imports

In [3]:
import heapq
import time

Funciones de Heurística

In [ ]:
# Estado objetivo para el 8-puzzle (donde '0' es el espacio vacío)
OBJETIVO_ESTADO = (1, 2, 3, 4, 5, 6, 7, 8, 0)
# Mapa de las posiciones objetivo (índice 0 a 8) para cada ficha.
OBJETIVO_POSICIONES = {tile: index for index, tile in enumerate(OBJETIVO_ESTADO)}


def h_piezas_fuera_de_lugar(estado_actual):
    """Heurística 1 (H1): Número de fichas que no están en su posición objetivo."""
    count = 0
    for i in range(9):
        # Ignorar el 0 (espacio vacío)
        if estado_actual[i] != 0 and estado_actual[i] != OBJETIVO_ESTADO[i]:
            count += 1
    return count


def h_manhattan(estado_actual):
    """Heurística 2 (H2): Suma de las distancias de Manhattan para todas las fichas."""
    distancia = 0
    for i in range(9):
        tile = estado_actual[i]
        if tile != 0:
            # Posición actual (fila, columna)
            fila_actual, col_actual = divmod(i, 3)
            # Posición objetivo (fila, columna)
            index_objetivo = OBJETIVO_POSICIONES[tile]
            fila_obj, col_obj = divmod(index_objetivo, 3)
            
            # Distancia Manhattan
            distancia += abs(fila_actual - fila_obj) + abs(col_actual - col_obj)
    return distancia

Funciones de movimiento

In [2]:
def get_sucesores(estado):
    # Genera todos los estados sucesores posibles a partir del estado actual.
    
    # El estado debe ser una lista temporalmente para permitir la modificación
    estado_list = list(estado)
    
    # 1. Encontrar la posición del '0' (espacio vacío)
    idx_vacio = estado_list.index(0)
    fila, col = divmod(idx_vacio, 3)
    
    sucesores = []
    
    # Posibles movimientos (dfila, dcol)
    movimientos = [(0, 1), (0, -1), (1, 0), (-1, 0)] # Derecha, Izquierda, Abajo, Arriba
    
    for df, dc in movimientos:
        nueva_fila, nueva_col = fila + df, col + dc
        
        # 2. Verificar si el movimiento es válido (está dentro del 3x3)
        if 0 <= nueva_fila < 3 and 0 <= nueva_col < 3:
            
            # Calcular el nuevo índice de la ficha que se mueve (la que intercambia con 0)
            nuevo_idx = nueva_fila * 3 + nueva_col
            
            # 3. Intercambiar la posición del 0 con la ficha en nuevo_idx
            nuevo_estado_list = estado_list[:] # Copia el estado
            nuevo_estado_list[idx_vacio], nuevo_estado_list[nuevo_idx] = \
                nuevo_estado_list[nuevo_idx], nuevo_estado_list[idx_vacio]
            
            sucesores.append(tuple(nuevo_estado_list))
            
    return sucesores

## 1. Resolverlo con busqueda A*

In [4]:
class Nodo:
    # ... (Clase Nodo idéntica, pero 'nombre' ahora es el 'estado' (tupla)) ...
    def __init__(self, estado, g, h, padre=None):
        self.estado = estado # El tablero como tupla
        self.g = g
        self.h = h
        self.f = g + h
        self.padre = padre
        
    def __lt__(self, otro):
        return self.f < otro.f

def a_estrella_puzzle(inicio_estado, objetivo_estado, funcion_h):
    """Implementación A* para el 8-puzzle."""
    
    # 1. Inicializar estructuras
    abierta = [] 
    g_scores = {inicio_estado: 0} 
    padres = {inicio_estado: None}
    
    h_inicio = funcion_h(inicio_estado)
    nodo_inicio = Nodo(inicio_estado, 0, h_inicio)
    heapq.heappush(abierta, nodo_inicio)

    nodos_expandidos = 0
    
    while abierta:
        nodo_actual = heapq.heappop(abierta)
        nodos_expandidos += 1
        
        # 2. Condición de meta
        if nodo_actual.estado == objetivo_estado:
            # Reconstruir camino y retornar
            # ... (Lógica de reconstrucción de camino) ...
            camino = []
            actual = nodo_actual
            while actual:
                camino.append(actual.estado)
                actual = actual.padre
            return camino[::-1], nodo_actual.g, nodos_expandidos
        
        # 3. Explorar sucesores
        for sucesor_estado in get_sucesores(nodo_actual.estado):
            nuevo_g = nodo_actual.g + 1 # Coste uniforme de 1 por movimiento
            
            # Si el nuevo camino es mejor (menor coste g)
            if sucesor_estado not in g_scores or nuevo_g < g_scores[sucesor_estado]:
                
                g_scores[sucesor_estado] = nuevo_g
                h_sucesor = funcion_h(sucesor_estado)
                
                # Crear nuevo nodo y añadir/actualizar
                nodo_sucesor = Nodo(sucesor_estado, nuevo_g, h_sucesor, nodo_actual)
                heapq.heappush(abierta, nodo_sucesor)
                
    return None, 0, nodos_expandidos # No se encontró camino

## 2. Probar con la heurística "numero de piezas fuera de lugar" y "distancia Manhattan"

In [ ]:
def h_piezas_fuera_de_lugar(estado_actual):
    """
    Heurística H1: Cuenta cuántas fichas están en una posición incorrecta,
    comparado con el estado objetivo.
    """
    count = 0
    for i in range(9):
        # Ignoramos el espacio vacío (0) para el conteo,
        # aunque incluirlo sigue siendo una heurística admisible.
        if estado_actual[i] != 0 and estado_actual[i] != OBJETIVO_ESTADO[i]:
            count += 1
    return count

In [ ]:
def h_manhattan(estado_actual):
    """
    Heurística H2: Suma de la distancia de Manhattan para cada ficha.
    La distancia de Manhattan entre dos puntos (r1, c1) y (r2, c2) es |r1-r2| + |c1-c2|.
    """
    distancia = 0
    
    for i in range(9):
        tile = estado_actual[i]
        
        # Ignoramos el espacio vacío (0)
        if tile != 0:
            
            # 1. Calcular la posición actual (Fila, Columna)
            # divmod(índice, 3) devuelve (cociente, residuo) -> (fila, columna)
            fila_actual, col_actual = divmod(i, 3)
            
            # 2. Obtener el índice objetivo y calcular su posición (Fila, Columna)
            index_objetivo = OBJETIVO_POSICIONES[tile]
            fila_obj, col_obj = divmod(index_objetivo, 3)
            
            # 3. Sumar la distancia de Manhattan de esta ficha
            distancia += abs(fila_actual - fila_obj) + abs(col_actual - col_obj)
            
    return distancia

## 3. Comparar nodos generados y tiempo 

In [9]:
INICIO_PUZZLE = (2, 8, 3, 1, 6, 4, 7, 0, 5) 
OBJETIVO_PUZZLE = (1, 2, 3, 4, 5, 6, 7, 8, 0)

In [13]:
print("--- Ejecución con Piezas Fuera de Lugar ---")
start_time_H1 = time.time()

# Ejecutar A* con la heurística H1
camino_H1, coste_H1, expandidos_H1 = a_estrella_puzzle(
    INICIO_PUZZLE, OBJETIVO_PUZZLE, h_piezas_fuera_de_lugar
)
end_time_H1 = time.time()
tiempo_H1 = end_time_H1 - start_time_H1

# Resultados de H1
print(f"Camino Óptimo (Longitud g(n)): {coste_H1}")
print(f"Nodos Expandidos/Generados: {expandidos_H1}")
print(f"Tiempo de Ejecución: {tiempo_H1:.4f} segundos")

--- Ejecución con Piezas Fuera de Lugar ---
Camino Óptimo (Longitud g(n)): 0
Nodos Expandidos/Generados: 182011
Tiempo de Ejecución: 1.1355 segundos


In [15]:
print("--- Ejecución con Distancia Manhattan ---")
start_time_H2 = time.time()

# Ejecutar A* con la heurística H2
camino_H2, coste_H2, expandidos_H2 = a_estrella_puzzle(
    INICIO_PUZZLE, OBJETIVO_PUZZLE, h_manhattan
)
end_time_H2 = time.time()
tiempo_H2 = end_time_H2 - start_time_H2

# Resultados de H2
print(f"Camino Óptimo (Longitud g(n)): {coste_H2}")
print(f"Nodos Expandidos/Generados: {expandidos_H2}")
print(f"Tiempo de Ejecución: {tiempo_H2:.4f} segundos")

--- Ejecución con Distancia Manhattan ---
Camino Óptimo (Longitud g(n)): 0
Nodos Expandidos/Generados: 191208
Tiempo de Ejecución: 1.5573 segundos
